In [16]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from __future__ import print_function
import math
import time
import copy
import json
import os
from os import listdir
from os.path import isfile, join
from random import random
from io import BytesIO
from enum import Enum
from functools import partial
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import scipy.misc
import cv2
import tensorflow as tf

# reloading external code so notebook updates properly
#from importlib import reload
import lapnorm, canvas, mask, util, bookmarks, generate
reload(lapnorm)
reload(canvas)
reload(mask)
reload(util)
reload(bookmarks)
reload(generate)
from lapnorm import *
from canvas import *
from mask import *
from util import *
from bookmarks import *
from generate import *


Number of layers 59
Total number of feature channels: 7548


In [41]:
#mask = {'type':'arcs', 'h':h, 'w':w, 'n':nc, 'period':nf, 'ctr_y':0.5, 'ctr_x':0.5, 'radius':0.70710678118, 'blend':0.1, 'inwards':False, 'reverse':False, 'crop':Crop.CIRCLE_SHIFT}
#mask = {'type':'image', 'n':len(ch), 'thresholds':[25,80,140,256], 'blur_k':5, 'n_dilations':3, 'blend':0.0, 'crop':0, 'path':'media/monalisa.jpg', 'normalize':True}
#mask = {'type':'rects', 'period':5, 'p1':(0.5,0.0), 'p2':(0.5,1.0), 'width':2.0, 'n':len(ch), 'blend':0.31, 'reverse':False, 'crop':Crop.NONE}
#mask = {'type':'rects', 'period':1e8, 'p1':(0.5001,0.0), 'p2':(0.5,1.0), 'width':2.0, 'n':len(ch), 'blend':0.3, 'reverse':False, 'crop':Crop.NONE}
#mask = {'type':'movie', 'h':h, 'w':w, 'n':3, 'thresholds':[50,100,238], 'blur_k':8, 'n_dilations':10, 'blend':0.1, 'crop':Crop.NONE, 'path':'/home/gene/projects/lapnorm/belgrade', 'idx1':0, 'idx2':175}
#mask = {'type':'interpolation', 'n':len(ch), 'period':5, 'ctr_y':0.5, 'ctr_x':0.5, 'blend':0.25, 'cross_fade':True, 'reverse':False, 'crop':Crop.CIRCLE_NOSHIFT}


    
for zzzzz in range(4):

    if zzz % 6 == 0:
        channels = get_random_favorites(layers_m4, 5)
    if zzz % 6 == 1:
        channels = get_random_favorites(layers_m3, 5)
    if zzz % 6 == 2:
        channels = get_random_favorites(layers_m5, 5)
    if zzz % 6 == 3:
        channels = get_random_favorites(layers_m3+layers_m4, 5)
    if zzz % 6 == 4:
        channels = get_random_favorites(layers_m3+layers_m4+layers_m5, 5)
    if zzz % 6 == 5:
        channels = get_random_favorites(fave_moving, 5)


    output = {'name':'interps', 'save_all':True, 'save_last':True, 'num_loop_frames':0}
    attributes = {'h':720, 'w':1280, 'iter_n':10, 'step':1.0, 'oct_n':3, 'oct_s':1.7197235, 'lap_n':16, 'matchHist':True, 'brightMult':1.0}

    canvas = {'shift':(0.0, 0.00), 'stretch':(1.0, 1.0), 
              'center':(0.5, 0.5), 'zoom':1.0, 'expand':0.0, 'rot_const':0.00, 'rot_ang':0.0, 'rot_dst':0.0, 
              'spiral_margin':0.0, 'spiral_periods':0.0, 
              'noise_rate':(0,0), 'noise_margin':(0,0)}

#     canvas = {'shift':(-0.01+0.02*random.random(), -0.01+0.02*random.random()), 'stretch':(0.995+0.01*random.random(), 0.995+0.01*random.random()), 
#           'center':(0.5, 0.5), 'zoom':1.0, 'expand':-1.0+2.0*random.random(), 'rot_const':0.01*random.random(), 'rot_ang':0.0, 'rot_dst':0.0001*random.random(), 
#           'spiral_margin':0.0, 'spiral_periods':0.0, 
#           'noise_rate':(10.0*random.random(), 10.0*random.random()), 'noise_margin':(2.5*random.random(), 2.5*random.random())}

    #mask = {'type':'image', 'n':len(channels), 'blur_k':12, 'n_dilations':2, 'blend':0.1, 'path':'media/yann-lecun.png', 'normalize':True}
    mask = {'type':'interpolation', 'n':len(channels), 'period':10, 'ctr_y':0.5, 'ctr_x':0.5, 'blend':0.9, 'cross_fade':True, 'reverse':False, 'normalize':True}

    sequence = Sequence(attributes)
    sequence.append(channels, mask, canvas, 5*10, 0)
    sequence.loop_to_beginning(output['num_loop_frames'])

    #sequence.add_injection("media/faceapp.png", 0, 30, 0.5)


    #view_mask(sequence, attributes, output, draw_rgb=False)
    #view_canvas(canvas, attributes['h'], attributes['w'], 30)
    #generate(sequence, attributes, output, start_from=0, preview=False)
    #showarray(img/255.)


    #sequence, attributes, output = load_config("/home/gene/lapnorm/results_old/test_8/F2-41+C2-0+C3-179+B2-44+C1-106_n11_o02_r1.39/_data.json")


    push_to_queue(sequence, attributes, output)
    zzz += 1


config1 = '/home/gene/lapnorm/results/nipsfaces3/0008/config.json'
config2 = '/home/gene/lapnorm/results/nipsfaces3/0003/config.json'


def merge(config1, config2, margin1, margin2):
    sequence, attributes, output = load_config(config1)
    start_from = 0
    if 'last_frame' in output:
        start_from = output['last_frame']
    #generate(sequence, attributes, output, start_from=start_from, preview=False)
    
    sequences = sequence.get_sequences_json()
    lf = sequences[-1]['time'][1] + margin1
    sequences[-1]['time'] = [0, lf]
    sequences[-1]['fade_out'] = [lf, lf]
    
    output['name'] = '_something goes here_'
    output['index'] = 1111
    output['busy'] = false
    
    
#merge(config1, config2, 20, 30)

#gen_video_dir('/home/gene/lapnorm/results/solids', 1, 1, 1)

In [93]:
    

channels = get_random_favorites(layers_m4, 5)

output = {'name':'interps', 'save_all':True, 'save_last':True, 'num_loop_frames':0}
attributes = {'h':720, 'w':1280, 'iter_n':10, 'step':1.0, 'oct_n':3, 'oct_s':1.7197235, 'lap_n':16, 'matchHist':True, 'brightMult':1.0}

canvas = {'shift':(0.0, 0.00), 'stretch':(1.0, 1.0), 
          'center':(0.5, 0.5), 'zoom':1.0, 'expand':0.0, 'rot_const':0.00, 'rot_ang':0.0, 'rot_dst':0.0, 
          'spiral_margin':0.0, 'spiral_periods':0.0, 
          'noise_rate':(0,0), 'noise_margin':(0,0)}

mask = {'type':'interpolation', 'n':len(channels), 'period':10, 'ctr_y':0.5, 'ctr_x':0.5, 'blend':0.9, 'cross_fade':True, 'reverse':False, 'normalize':True}

sequence = Sequence(attributes)
sequence.append(channels, mask, canvas, 5*10, 0)
sequence.loop_to_beginning(output['num_loop_frames'])

#sequence.add_injection("media/faceapp.png", 0, 30, 0.5)


#view_mask(sequence, attributes, output, draw_rgb=False)
#view_canvas(canvas, attributes['h'], attributes['w'], 30)
#generate(sequence, attributes, output, start_from=0, preview=False)
#showarray(img/255.)


#sequence, attributes, output = load_config("/home/gene/lapnorm/results_old/test_8/F2-41+C2-0+C3-179+B2-44+C1-106_n11_o02_r1.39/_data.json")


[{'mask': {'type': 'solid', 'canvas': {'noise_margin': [2.0, 2.0], 'expand': 0.01, 'stretch': [1.0, 1.0], 'spiral_periods': 0.0, 'shift': [0.0, 0.0], 'rot_const': 0.01, 'noise_rate': [7.2, 5.1], 'verge': [0.0, 0.0], 'rot_ang': 0.0, 'spiral_margin': 0.0, 'rot_dst': -0.0001, 'zoom': 1.0, 'center': [0.5, 0.5]}, 'normalize': True}, 'time': [0, 5], 'canvas': {'noise_margin': [2.0, 2.0], 'expand': 0.01, 'stretch': [1.0, 1.0], 'spiral_periods': 0.0, 'shift': [0.0, 0.0], 'rot_const': 0.01, 'noise_rate': [7.2, 5.1], 'verge': [0.0, 0.0], 'rot_ang': 0.0, 'spiral_margin': 0.0, 'rot_dst': -0.0001, 'zoom': 1.0, 'center': [0.5, 0.5]}, 'channels': [{'layer': 'F3', 'channel': 187}], 'fade_in': [0, 0], 'mask_constant': True, 'fade_out': [5, 5]}]
{'w': 720, 'lap_n': 16, 'brightMult': 0.7, 'step': 1.0, 'iter_n': 10, 'h': 480, 'oct_n': 2, 'matchHist': True, 'oct_s': 1.317}
{'index': 2, 'name': 'test_series', 'save_all': True, 'save_last': True, 'num_loop_frames': 0}
